# Ejecutando el modelo como un MaaS (Model as a Service)

MLflow nos da la opción de "servir" nuestro modelo ejecutandolo en un servidor (en este caso local), e interactuar con él como si fuera una REST API, a la cual se le hacemos requests con nuestros datos o inputs, y que responde con la predicción del modelo. Esto se puede hacer de manera sencilla y a continuación se presentarán los pasos para hacerlo.

## Configurando variables de ambiente

Antes de poder recuperar nuestro modelo del servidor de MLflow, es necesario configurar el URI del mismo, exportandolo como variable de ambiente llamada MLFLOW_TRACKING_URI. En nuestro caso, podemos hacer esto ejecutando la siguiente linea en la terminal de nuestro sistema:

```bash
export MLFLOW_TRACKING_URI=https://dagshub.com/AAA-Energia/energy-demand-prediction.mlflow
```

En este caso, `https://dagshub.com/AAA-Energia/energy-demand-prediction.mlflow` es nuestro URI de MLflow, donde estan almacenados nuestros runs y modelos, pero puede ser diferente para cada proyecto.

## Recuperando y sirviendo el modelo

Una vez configurada nuestra variable de ambiente, podemos proceder a levantar nuestro servidor local ejecutando el siguiente comando en nuestra terminal:

```bash
mlflow models serve -m models:/{nombre_modelo}@champion -p {puerto} --no-conda
```

El parámetro `{puerto}` simplemente se refiere al puerto que queremos utilizar para nuestro servidor local, como por ejemplo el 5000. Por otro lado, el `{nombre_modelo}` representa el modelo que vamos a utilizar en nuestro servidor, tal y como está almacenado en el registro de modelos de MLflow. En el caso de nuestro proyecto, estos pueden ser:

- `Prophet_Model`
- `random_forest_model`
- `Gradient_Boosting_Model`

Para poner un ejemplo, supongamos que queremos levantar un servidor local con nuestro modelo Prophet en el puerto, 5000, en este caso, ejecutariamos el siguiente comando en nuestra terminal:

```bash
mlflow models serve -m models:/Prophet_Model@champion -p 5000 --no-conda
```

Lo cual nos imprimiría lo siguiete en la terminanal, dejandonos saber la dirección en la cual nuestro modelo está "escuchando":
```bash
[2024-05-31 15:55:36 -0700] [4713] [INFO] Starting gunicorn 22.0.0
[2024-05-31 15:55:36 -0700] [4713] [INFO] Listening at: http://127.0.0.1:5000 (4713)
[2024-05-31 15:55:36 -0700] [4713] [INFO] Using worker: sync
[2024-05-31 15:55:36 -0700] [4715] [INFO] Booting worker with pid: 4715
```

Con esto, podemos hacer requests a nuestro modelo como si fuera una API REST, al URL especificado en la terminal.

De acuerdo con la [documentación de MLflow](https://mlflow.org/docs/latest/deployment/deploy-model-locally.html), nuestro servidor cuenta con los siguientes endpoints: 

- `/invocations`: el endpoint de inferencia de nuestro servidor. Recibe requests de tipo POST con el input del modelo, y regresa la predicción del mismo.
- `/ping`: para revisar el estado (health) de nuestro servidor.
- `/health`: mismo caso que el endpoint anterior.
- `/version`: regresa la version utilizada de MLflow.

En este caso, nuestro interés se enfoca en el primer endpoint, y a continuación se explicará el como hacer requests al mismo y obtener las predicciónes del modelo.

## Realizando peticiones a nuestro modelo

### Peticiones de tipo CSV

El endpoint de inferencia de nuestro servidor permite realizar las peticiones tanto en formato JSON como en formato CSV. En ambos casos, las peticiones se hacen de manera similar, cambiando simplemente el payload de las mismas.

Para el caso de las peticiones CSV, el payload puede ser simplemente un texto de tipo CSV, especificando con el header `Content-Type: text/csv` que se trata de este.

Veamos un ejemplo de esto utilizando nuestro modelo `random_forest_model`, para esto primero hay que ejecutar la siguiente línea en nuestra terminal, para levantar nuestro servidor local:

```bash
mlflow models serve -m models:/random_forest_model@champion -p 5000 --no-conda
```

Después, al armar nuestro requests con python y la librería `requests`, vemos que en la variable `payload` tenemos un string representando nuestro CSV con dos lineas: 

1. La primer linea contiene el nombre de las variables de entrada, las cuales son requeridas y deben tener la misma estructura (nombre de variables) que los datos con los cuales se entrenó el modelo, todos separados por comas.
2. La segunda linea son los valores de entrada para nuestro modelo, igualmente separados por comas.

Una vez hecho esto, especificamos los headers cmoo definimos anteriormente, realizamos la petición, y vemos como nuestro servidor responde con la predicción del modelo:

In [13]:
import requests
import json
import pandas as pd

base_url = "http://127.0.0.1:5000/"
url = base_url + "invocations"

payload = "tmax-cab(t-12),tmax-hmo(t-12),tmax-obr(t-12),tmax-lmo(t-12),tmax-cul(t-12),tmin-cab(t-12),tmin-hmo(t-12),tmin-obr(t-12),tmin-lmo(t-12),tmin-cul(t-12),prec_hmo_mm(t-12),prec_obr_mm(t-12),prec_lmo_mm(t-12),prec_cul_mm(t-12),lunes_festivo(t-12),martes_postfestivo(t-12),semana_santa(t-12),1_mayo(t-12),10_mayo(t-12),16_sep(t-12),2_nov.(t-12),pre-navidad_y_new_year(t-12),navidad_y_new_year(t-12),post-navidad_y_new_year(t-12),demanda_energia(t-12),tmax-cab(t-11),tmax-hmo(t-11),tmax-obr(t-11),tmax-lmo(t-11),tmax-cul(t-11),tmin-cab(t-11),tmin-hmo(t-11),tmin-obr(t-11),tmin-lmo(t-11),tmin-cul(t-11),prec_hmo_mm(t-11),prec_obr_mm(t-11),prec_lmo_mm(t-11),prec_cul_mm(t-11),lunes_festivo(t-11),martes_postfestivo(t-11),semana_santa(t-11),1_mayo(t-11),10_mayo(t-11),16_sep(t-11),2_nov.(t-11),pre-navidad_y_new_year(t-11),navidad_y_new_year(t-11),post-navidad_y_new_year(t-11),demanda_energia(t-11),tmax-cab(t-10),tmax-hmo(t-10),tmax-obr(t-10),tmax-lmo(t-10),tmax-cul(t-10),tmin-cab(t-10),tmin-hmo(t-10),tmin-obr(t-10),tmin-lmo(t-10),tmin-cul(t-10),prec_hmo_mm(t-10),prec_obr_mm(t-10),prec_lmo_mm(t-10),prec_cul_mm(t-10),lunes_festivo(t-10),martes_postfestivo(t-10),semana_santa(t-10),1_mayo(t-10),10_mayo(t-10),16_sep(t-10),2_nov.(t-10),pre-navidad_y_new_year(t-10),navidad_y_new_year(t-10),post-navidad_y_new_year(t-10),demanda_energia(t-10),tmax-cab(t-9),tmax-hmo(t-9),tmax-obr(t-9),tmax-lmo(t-9),tmax-cul(t-9),tmin-cab(t-9),tmin-hmo(t-9),tmin-obr(t-9),tmin-lmo(t-9),tmin-cul(t-9),prec_hmo_mm(t-9),prec_obr_mm(t-9),prec_lmo_mm(t-9),prec_cul_mm(t-9),lunes_festivo(t-9),martes_postfestivo(t-9),semana_santa(t-9),1_mayo(t-9),10_mayo(t-9),16_sep(t-9),2_nov.(t-9),pre-navidad_y_new_year(t-9),navidad_y_new_year(t-9),post-navidad_y_new_year(t-9),demanda_energia(t-9),tmax-cab(t-8),tmax-hmo(t-8),tmax-obr(t-8),tmax-lmo(t-8),tmax-cul(t-8),tmin-cab(t-8),tmin-hmo(t-8),tmin-obr(t-8),tmin-lmo(t-8),tmin-cul(t-8),prec_hmo_mm(t-8),prec_obr_mm(t-8),prec_lmo_mm(t-8),prec_cul_mm(t-8),lunes_festivo(t-8),martes_postfestivo(t-8),semana_santa(t-8),1_mayo(t-8),10_mayo(t-8),16_sep(t-8),2_nov.(t-8),pre-navidad_y_new_year(t-8),navidad_y_new_year(t-8),post-navidad_y_new_year(t-8),demanda_energia(t-8),tmax-cab(t-7),tmax-hmo(t-7),tmax-obr(t-7),tmax-lmo(t-7),tmax-cul(t-7),tmin-cab(t-7),tmin-hmo(t-7),tmin-obr(t-7),tmin-lmo(t-7),tmin-cul(t-7),prec_hmo_mm(t-7),prec_obr_mm(t-7),prec_lmo_mm(t-7),prec_cul_mm(t-7),lunes_festivo(t-7),martes_postfestivo(t-7),semana_santa(t-7),1_mayo(t-7),10_mayo(t-7),16_sep(t-7),2_nov.(t-7),pre-navidad_y_new_year(t-7),navidad_y_new_year(t-7),post-navidad_y_new_year(t-7),demanda_energia(t-7),tmax-cab(t-6),tmax-hmo(t-6),tmax-obr(t-6),tmax-lmo(t-6),tmax-cul(t-6),tmin-cab(t-6),tmin-hmo(t-6),tmin-obr(t-6),tmin-lmo(t-6),tmin-cul(t-6),prec_hmo_mm(t-6),prec_obr_mm(t-6),prec_lmo_mm(t-6),prec_cul_mm(t-6),lunes_festivo(t-6),martes_postfestivo(t-6),semana_santa(t-6),1_mayo(t-6),10_mayo(t-6),16_sep(t-6),2_nov.(t-6),pre-navidad_y_new_year(t-6),navidad_y_new_year(t-6),post-navidad_y_new_year(t-6),demanda_energia(t-6),tmax-cab(t-5),tmax-hmo(t-5),tmax-obr(t-5),tmax-lmo(t-5),tmax-cul(t-5),tmin-cab(t-5),tmin-hmo(t-5),tmin-obr(t-5),tmin-lmo(t-5),tmin-cul(t-5),prec_hmo_mm(t-5),prec_obr_mm(t-5),prec_lmo_mm(t-5),prec_cul_mm(t-5),lunes_festivo(t-5),martes_postfestivo(t-5),semana_santa(t-5),1_mayo(t-5),10_mayo(t-5),16_sep(t-5),2_nov.(t-5),pre-navidad_y_new_year(t-5),navidad_y_new_year(t-5),post-navidad_y_new_year(t-5),demanda_energia(t-5),tmax-cab(t-4),tmax-hmo(t-4),tmax-obr(t-4),tmax-lmo(t-4),tmax-cul(t-4),tmin-cab(t-4),tmin-hmo(t-4),tmin-obr(t-4),tmin-lmo(t-4),tmin-cul(t-4),prec_hmo_mm(t-4),prec_obr_mm(t-4),prec_lmo_mm(t-4),prec_cul_mm(t-4),lunes_festivo(t-4),martes_postfestivo(t-4),semana_santa(t-4),1_mayo(t-4),10_mayo(t-4),16_sep(t-4),2_nov.(t-4),pre-navidad_y_new_year(t-4),navidad_y_new_year(t-4),post-navidad_y_new_year(t-4),demanda_energia(t-4),tmax-cab(t-3),tmax-hmo(t-3),tmax-obr(t-3),tmax-lmo(t-3),tmax-cul(t-3),tmin-cab(t-3),tmin-hmo(t-3),tmin-obr(t-3),tmin-lmo(t-3),tmin-cul(t-3),prec_hmo_mm(t-3),prec_obr_mm(t-3),prec_lmo_mm(t-3),prec_cul_mm(t-3),lunes_festivo(t-3),martes_postfestivo(t-3),semana_santa(t-3),1_mayo(t-3),10_mayo(t-3),16_sep(t-3),2_nov.(t-3),pre-navidad_y_new_year(t-3),navidad_y_new_year(t-3),post-navidad_y_new_year(t-3),demanda_energia(t-3),tmax-cab(t-2),tmax-hmo(t-2),tmax-obr(t-2),tmax-lmo(t-2),tmax-cul(t-2),tmin-cab(t-2),tmin-hmo(t-2),tmin-obr(t-2),tmin-lmo(t-2),tmin-cul(t-2),prec_hmo_mm(t-2),prec_obr_mm(t-2),prec_lmo_mm(t-2),prec_cul_mm(t-2),lunes_festivo(t-2),martes_postfestivo(t-2),semana_santa(t-2),1_mayo(t-2),10_mayo(t-2),16_sep(t-2),2_nov.(t-2),pre-navidad_y_new_year(t-2),navidad_y_new_year(t-2),post-navidad_y_new_year(t-2),demanda_energia(t-2),tmax-cab(t-1),tmax-hmo(t-1),tmax-obr(t-1),tmax-lmo(t-1),tmax-cul(t-1),tmin-cab(t-1),tmin-hmo(t-1),tmin-obr(t-1),tmin-lmo(t-1),tmin-cul(t-1),prec_hmo_mm(t-1),prec_obr_mm(t-1),prec_lmo_mm(t-1),prec_cul_mm(t-1),lunes_festivo(t-1),martes_postfestivo(t-1),semana_santa(t-1),1_mayo(t-1),10_mayo(t-1),16_sep(t-1),2_nov.(t-1),pre-navidad_y_new_year(t-1),navidad_y_new_year(t-1),post-navidad_y_new_year(t-1),demanda_energia(t-1),tmax-cab(t),tmax-hmo(t),tmax-obr(t),tmax-lmo(t),tmax-cul(t),tmin-cab(t),tmin-hmo(t),tmin-obr(t),tmin-lmo(t),tmin-cul(t),prec_hmo_mm(t),prec_obr_mm(t),prec_lmo_mm(t),prec_cul_mm(t),lunes_festivo(t),martes_postfestivo(t),semana_santa(t),1_mayo(t),10_mayo(t),16_sep(t),2_nov.(t),pre-navidad_y_new_year(t),navidad_y_new_year(t),post-navidad_y_new_year(t)\r\n39,39,38,35,38,22,18,22,21,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3049,39,39,38,35,38,22,18,22,21,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2903,39,39,38,35,38,22,18,22,21,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2737,39,39,38,35,38,22,18,22,21,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2821,39,39,38,35,38,22,18,22,21,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2992,39,39,38,35,38,22,18,22,21,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3108,41,40,42,35,38,23,18,23,22,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3088,41,40,42,35,38,23,18,23,22,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2994,41,40,42,35,38,23,18,23,22,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2856,41,40,42,35,38,23,18,23,22,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2718,41,40,42,35,38,23,18,23,22,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2598,41,40,42,35,38,23,18,23,22,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2493,41,40,42,35,38,23,18,23,22,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
headers = {
  'Content-Type': 'text/csv'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"predictions": [2407.0154069925006]}


Podemos despues convertir nuestra respuesta a un dataframe para una mejor manipulación de ésta:

In [15]:
pd.DataFrame.from_dict(response.json())

,predictions
0,2407.015407


### Peticiones de tipo JSON

También podemos realizar peticiones a nuestro servidor con payloads de tipo JSON. Para esto, MLflow define en [su documentación](https://mlflow.org/docs/latest/deployment/deploy-model-locally.html#json-input), distintos formatos para las peticiones de tipo JSON. Veamos un ejemplo utilizando el formato `"dataframe_split"`.

Veamos ahora un ejemplo con nuestro modelo Prophet. Para comenzar, levantamos nuestro servidor local con el siguiente comando en la terminal:

```bash
mlflow models serve -m models:/Prophet_Model@champion -p 5000 --no-conda
```

De manera similar a la anterior, definimos ahora nuestro payload con la misma estructura que los datos de entrenamiento del modelo, generamos los headers y hacemos la petición. De nuevo, podemos ver que nuestro servidor regresa la predicción del modelo para los datos proporcionados:

In [22]:
from io import StringIO

payload = json.dumps({
  "dataframe_split": {
    "columns": ["ds", "y", "floor", "t", "y_scaled"],
    "data": [
      ["2024-01-01 00:00:00", 0, 0, 0, 0],
      ["2024-01-01 01:00:00", 0, 0, 0, 0],
      ["2024-01-01 02:00:00", 0, 0, 0, 0],
      ["2024-01-01 03:00:00", 0, 0, 0, 0],
      ["2024-01-01 04:00:00", 0, 0, 0, 0]
    ],
    "index": [0, 1, 2, 3, 4]
  }
})


headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"predictions": [{"ds": "2024-01-01T00:00:00", "trend": 3629.9759799274734, "yhat_lower": 2714.444743705902, "yhat_upper": 3250.5932041374367, "trend_lower": 3629.9759799274734, "trend_upper": 3629.9759799274734, "additive_terms": -660.7899979914624, "additive_terms_lower": -660.7899979914624, "additive_terms_upper": -660.7899979914624, "daily": 192.32154335947465, "daily_lower": 192.32154335947465, "daily_upper": 192.32154335947465, "weekly": -123.04622358707945, "weekly_lower": -123.04622358707945, "weekly_upper": -123.04622358707945, "yearly": -730.0653177638575, "yearly_lower": -730.0653177638575, "yearly_upper": -730.0653177638575, "multiplicative_terms": 0.0, "multiplicative_terms_lower": 0.0, "multiplicative_terms_upper": 0.0, "yhat": 2969.185981936011}, {"ds": "2024-01-01T01:00:00", "trend": 3629.99018863278, "yhat_lower": 2635.881548445359, "yhat_upper": 3108.664684327231, "trend_lower": 3629.99018863278, "trend_upper": 3629.99018863278, "additive_terms": -762.3343905715743, "

Finalmente, podemos procesar la respuesta de nuestro servidor utilizando pandas para generar un dataframe, y poder analizar mejor las predicciones de nuestro modelo:

In [26]:
json_string = json.dumps(response.json()['predictions'])

res_df = pd.read_json(StringIO(json_string))

res_df.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-01-01T00:00:00,3629.975980,2714.444744,3250.593204,3629.975980,3629.975980,-660.789998,-660.789998,-660.789998,192.321543,...,-123.046224,-123.046224,-123.046224,-730.065318,-730.065318,-730.065318,0,0,0,2969.185982
1,2024-01-01T01:00:00,3629.990189,2635.881548,3108.664684,3629.990189,3629.990189,-762.334391,-762.334391,-762.334391,82.618403,...,-114.962804,-114.962804,-114.962804,-729.989990,-729.989990,-729.989990,0,0,0,2867.655798
2,2024-01-01T02:00:00,3630.004397,2528.263499,3022.717981,3630.004397,3630.004397,-863.508669,-863.508669,-863.508669,-27.148226,...,-106.447237,-106.447237,-106.447237,-729.913206,-729.913206,-729.913206,0,0,0,2766.495728
3,2024-01-01T03:00:00,3630.018606,2442.158957,2954.248118,3630.018606,3630.018606,-931.708327,-931.708327,-931.708327,-104.310876,...,-97.562480,-97.562480,-97.562480,-729.834971,-729.834971,-729.834971,0,0,0,2698.310279
4,2024-01-01T04:00:00,3630.032815,2422.101805,2898.950214,3630.032815,3630.032815,-972.405036,-972.405036,-972.405036,-154.275729,...,-88.374018,-88.374018,-88.374018,-729.755289,-729.755289,-729.755289,0,0,0,2657.627779
